In [ ]:
%load_ext autoreload
%autoreload 2

### Download an example image

In [ ]:
import os
import requests

# Create the directory if it does not exist
examples_dir = "../instanseg/examples/"
if not os.path.exists(examples_dir):
    os.makedirs(examples_dir)

#This is the LuCa-7color_[13860,52919]_1x1 from Perkin Elmer CC-BY 4.0
download_url = "https://downloads.openmicroscopy.org/images/Vectra-QPTIFF/perkinelmer/PKI_fields/LuCa-7color_%5b13860,52919%5d_1x1component_data.tif"
local_file_path = os.path.join(examples_dir, "LuCa1.tif")

# Download the file using requests
response = requests.get(download_url, stream=True)
response.raise_for_status()
with open(local_file_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"File downloaded to {local_file_path}")

### Download pretrained models

In [ ]:
from instanseg import InstanSeg
from instanseg.utils.utils import show_images

instanseg_brightfield = InstanSeg("brightfield_nuclei", image_reader= "tiffslide", verbosity=1)
instanseg_fluorescence = InstanSeg("fluorescence_nuclei_and_cells")

**The easiest way to use the model is to call the eval method:**


In [ ]:
labeled_output = instanseg_brightfield.eval(image = "../instanseg/examples/HE_example.tif",
                                            save_output = True,
                                            save_overlay=True,)
                                            

**Most of the time, we need more granularity, so the inference can be broken down in smaller steps:**


In [ ]:
image_array, pixel_size = instanseg_brightfield.read_image("../instanseg/examples/HE_example.tif")

labeled_output, image_tensor  = instanseg_brightfield.eval_small_image(image_array, pixel_size)

display = instanseg_brightfield.display(image_tensor, labeled_output)

show_images(image_tensor,display, colorbar=False, titles = ["Original Image", "Image with segmentation"])


**InstanSeg is designed for the segmentation of cells and nuclei in multiplexed images too**

In [ ]:

image_array, pixel_size = instanseg_fluorescence.read_image("../instanseg/examples/LuCa1.tif")

labeled_output, image_tensor  = instanseg_fluorescence.eval_small_image(image_array, pixel_size)

display = instanseg_fluorescence.display(image_tensor, labeled_output)

show_images(display, colorbar=False)



**Your image is too large to fit on the GPU? You can run InstanSeg across the image tiles.**

*The whole image still has to fit in RAM using *eval_medium_image*


By default, InstanSeg will output both cells and nuclei. You can specifiy the target parameter to predict only cells or nuclei

In [ ]:
instanseg_fluorescence = InstanSeg("fluorescence_nuclei_and_cells")

image_array, pixel_size = instanseg_fluorescence.read_image("../instanseg/examples/LuCa1.tif")

labeled_output, image_tensor  = instanseg_fluorescence.eval_medium_image(image_array, pixel_size, target= "cells")

display = instanseg_fluorescence.display(image_tensor, labeled_output)

show_images(display, colorbar=False)


**Your image is too large to fit in RAM? You can still run InstanSeg across whole slide images**

This is still an experimental feature


In [ ]:
instanseg_brightfield.eval_whole_slide_image("../instanseg/examples/HE_Hamamatsu.tiff")

In [ ]:
from instanseg import InstanSeg
from instanseg.utils.utils import show_images

instanseg_brightfield = InstanSeg("brightfield_nuclei", image_reader= "tiffslide", verbosity=1)

image_array, pixel_size = instanseg_brightfield.read_image("../instanseg/examples/HE_example.tif")

labeled_output, image_tensor  = instanseg_brightfield.eval_small_image(image_array, pixel_size)

display = instanseg_brightfield.display(image_tensor, labeled_output)

show_images(image_tensor,display, colorbar=False, titles = ["Normalized Image", "Image with segmentation"])